In [6]:
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import metrics


def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir, f) for f in os.listdir(train_dir)]
    all_words = []
    for mail in emails:
        with open(mail) as m:
            for i, line in enumerate(m):
                if i == 2:
                    words = line.split()
                    all_words += words

    dictionary = Counter(all_words)
    dictionary = dictionary.most_common(3000)
    return dictionary


def extract_features(mail_dir):
    files = [os.path.join(mail_dir, fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files), 3000))
    docID = 0;
    for fil in files:
        with open(fil) as fi:
            for i, line in enumerate(fi):
                if i == 2:
                    words = line.split()
                    for word in words:
                        wordID = 0
                        for i, d in enumerate(dictionary):
                            if d[0] == word:
                                wordID = i
                                features_matrix[docID, wordID] = words.count(word) 
            docID = docID + 1
    return features_matrix

In [7]:
# Create a dictionary of words with its frequency

train_dir = r"C:\Users\L\Downloads\rdbmsia2\ling-spam\train-mails"
dictionary = make_Dictionary(train_dir)

# Prepare feature vectors per training mail and its labels

train_labels = np.zeros(702)
train_labels[351:701] = 1
train_matrix = extract_features(train_dir)

In [8]:
# Training SVM and Naive bayes classifier and its variants

model1 = LinearSVC()
model2 = MultinomialNB()

model1.fit(train_matrix, train_labels)
model2.fit(train_matrix, train_labels)

C:\Users\L\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


MultinomialNB()

In [12]:
# Test the unseen mails for Spam

test_dir = r"C:\Users\L\Downloads\rdbmsia2\ling-spam\test-mails"
test_matrix = extract_features(test_dir)
test_labels = np.zeros(260)
test_labels[130:260] = 1

result1 = model1.predict(test_matrix)
result2 = model2.predict(test_matrix)

In [25]:
print(metrics.confusion_matrix(test_labels, result1))
print(model1.score(test_matrix, test_labels)*100)
print(metrics.confusion_matrix(test_labels, result2))
print(model2.score(test_matrix, test_labels)*100)

[[123   7]
 [  5 125]]
95.38461538461539
[[117  13]
 [ 12 118]]
90.38461538461539
